![](../images/itam_logo.png)

M. Sc. Liliana Millán Núñez liliana.millan@itam.mx

Febrero 2021


## AWS with Python

In [16]:
import boto3

In [17]:
# saber qué versión de Boto estamos ocupando
print(boto3.__version__)

1.17.6


Si tienes tu archivo `~/.aws/credentials` puedes ocupar dentro del objeto `Session`, qué perfil vas a ocupar para conectarte a AWS, también es posible indicar la región y las credenciales asociadas, aunque esto **no** se recomienda, en este caso es mejor poner las llaves en plano en algún archivo `yaml` en tu computadora y leerlos desde ahí para que **no** subas tus llaves al repositorio!.

#### Con profile

In [18]:
ses = boto3.session.Session(profile_name='robper_dpa')

In [19]:
s3 = ses.resource('s3')

In [20]:
# listar los buckets que están en este perfil y región.
[bucket.name for bucket in s3.buckets.all()]

['dpa-robtest']

### Leyendo llaves de yaml

In [21]:
import yaml

In [22]:
# este archivo yaml vive en tu conf/local/credentials.yaml
# este va en src/utils general.py
def read_yaml_file(yaml_file):
    """ load yaml cofigurations """

    config = None
    try: 
        with open (yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Couldnt load the file')
    
    return config

In [25]:
# este va en src/utils general.py
def get_s3_credentials(credentials_file):
    credentials = read_yaml_file(credentials_file)
    s3_creds = credentials['s3']

    return s3_creds

In [27]:
#s3_creds = get_s3_credentials("../../conf/local/credentials.yaml")
s3_creds = get_s3_credentials("/home/silil/Documents/itam/metodos_gran_escala/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

##### Listado de solo una carpeta en específico

In [28]:
bucket = "metodos-gran-escala"
key = "northwind"

In [29]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']

In [30]:
objects

[{'Key': 'northwind/customers.csv',
  'LastModified': datetime.datetime(2019, 2, 27, 1, 57, 6, tzinfo=tzutc()),
  'ETag': '"5f42cfb8c6ce316281596336f87f37bb"',
  'Size': 11558,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/employees.csv',
  'LastModified': datetime.datetime(2019, 2, 27, 1, 57, 6, tzinfo=tzutc()),
  'ETag': '"ce832fe36e34dd7942c322322500fef9"',
  'Size': 4059,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/order_details.csv',
  'LastModified': datetime.datetime(2019, 3, 6, 1, 57, 10, tzinfo=tzutc()),
  'ETag': '"f493c58c1cb13b6e9888dd091c6d3f10"',
  'Size': 40622,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/orders.csv',
  'LastModified': datetime.datetime(2019, 3, 6, 1, 56, 52, tzinfo=tzutc()),
  'ETag': '"cfa9eaf9c9e12e08e617747cf43a3921"',
  'Size': 98517,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/products.csv',
  'LastModified': datetime.datetime(2019, 2, 27, 1, 57, 6, tzinfo=tzutc()),
  'ETag': '"4e9ef122f1fe7673e1b0d3643e995d17"',
  'Size':

### Creación de un bucket

+ Por default los *buckets* son creados en la región *US East (N. Virginia)* `us-east-1`, necesitarás definir en qué región quieres crear el *bucket*. 
+ Los nombres de buckets **no** pueden llevar guiónes bajos, te marcará que el nombre es inválido, en lugar de los guiones bajos puedes ocupar guiónes altos.

In [31]:
#crear un bucket
bucket_name = 'test-aws-boto-clase-dpa'

s3.create_bucket(Bucket=bucket_name, 
                 CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
                ACL='private')

{'ResponseMetadata': {'RequestId': '1E6F57522E9E4F2F',
  'HostId': 'M5I2f/j1rk9RwxQ8tB1S1w3e/PYDFPiM2VFcZFTaRV0In0PPFS6h5NAEOD+g+ROuKYNQdIxDrw4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'M5I2f/j1rk9RwxQ8tB1S1w3e/PYDFPiM2VFcZFTaRV0In0PPFS6h5NAEOD+g+ROuKYNQdIxDrw4=',
   'x-amz-request-id': '1E6F57522E9E4F2F',
   'date': 'Fri, 12 Feb 2021 01:25:30 GMT',
   'location': 'http://test-aws-boto-clase-dpa.s3.amazonaws.com/',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': 'http://test-aws-boto-clase-dpa.s3.amazonaws.com/'}

### Guardar archivos en el bucket

In [1]:
# subiremos un archivo csv de ejemplo: pollution con ~43k registros 
#con observaciones de contaminación del aire en una ciudad de China. 
file_to_upload = '/home/silil/Documents/itam/mineria_datos_licenciatura/data/beijing_pollution/pollution.csv'
file_name = file_to_upload.split(sep='/')[-1]

# accedemos a client desde el resource
#s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)

In [42]:
# si están accediendo a través de client es asi
s3.upload_file(file_to_upload, bucket_name, file_name)

### Acceder al bucket

In [46]:
# acceder al bucket
# este es con el resource 
my_bucket = s3.Bucket(bucket_name)

for element in my_bucket.objects.all():
    print(element)

s3.ObjectSummary(bucket_name='test-aws-boto-clase-dpa', key='pollution.csv')


### Bajar archivos del bucket

In [47]:
path_to_download = '/home/silil/Documents/itam/metodos_gran_escala/data-product-architecture/' + file_name

# desde resource 
s3.meta.client.download_file(bucket_name, file_name, path_to_download)

# desde cliente
#s3.download_file(bucket_name, file_name, path_to_download)

### Borrar archivos del bucket

In [48]:
# si es resource
s3.Object(bucket_name, file_name).delete()

{'ResponseMetadata': {'RequestId': 'CB08B3557E392871',
  'HostId': 'P0OqdgQ/AECyp5xT5OIXAzR+/kE2QafNDL6yjj5H6gE1fs1hUxOqlGiZ4NyL407qQcrYGlaEeic=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'P0OqdgQ/AECyp5xT5OIXAzR+/kE2QafNDL6yjj5H6gE1fs1hUxOqlGiZ4NyL407qQcrYGlaEeic=',
   'x-amz-request-id': 'CB08B3557E392871',
   'date': 'Fri, 12 Feb 2021 01:34:07 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Borrar el bucket

In [49]:
# resource
s3.Bucket(bucket_name).delete()

{'ResponseMetadata': {'RequestId': 'F8D0799D8B90BE77',
  'HostId': '1f1e8eHXPuAEoD/muruOfYXvqKE388ajSS+JsBg1Hc8sEW6SdQWPes2vErk0Z18UMivO61K8jNw=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '1f1e8eHXPuAEoD/muruOfYXvqKE388ajSS+JsBg1Hc8sEW6SdQWPes2vErk0Z18UMivO61K8jNw=',
   'x-amz-request-id': 'F8D0799D8B90BE77',
   'date': 'Fri, 12 Feb 2021 01:35:04 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Referecias 

+ [S3 boto3 API Reference](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html)